# Amperon Data Engineering Take Home Assignment

In [1]:
import os
from ipywidgets import interact, widgets
from tomorrow.db import get_connection, execute_data_query
import pandas as pd
import matplotlib.pyplot as plt
import warnings
# Suppress all warnings
warnings.filterwarnings ('ignore')

In [2]:
# Bring in DB credentials from env
PGHOST = os.environ.get('PGHOST')
PGPORT = os.environ.get('PGPORT')
PGUSER = os.environ.get('PGUSER')
PGPASSWORD = os.environ.get('PGPASSWORD')
PGDATABASE = os.environ.get('PGDATABASE')

env_vars = [PGHOST, PGPORT, PGUSER, PGPASSWORD, PGDATABASE ]

# Check for missing env values. 
if any([ v is None for v in env_vars]):
    raise("Missing DB credentials.")

In [3]:
# Create Connection object
db_conn = get_connection()

Connecting to DB


In [4]:
# Define queries
hitorical_query = """ select * from historical_weather """
forecast_query  = """ select * from forecast_weather """

In [5]:
# Pull Data
historical_data = execute_data_query(db_conn, hitorical_query)
forecast_data = execute_data_query(db_conn, forecast_query)


In [6]:
# Preview Data
historical_data.head()

,id,datetime,field,value
0,1,2024-11-17 05:00:00,cloudBase,NaN
1,1,2024-11-17 05:00:00,cloudCeiling,NaN
2,1,2024-11-17 05:00:00,cloudCover,7.000
3,1,2024-11-17 05:00:00,dewPoint,67.330
4,1,2024-11-17 05:00:00,evapotranspiration,0.001


In [7]:
forecast_data.head()

,id,datetime,field,value
0,1,2024-11-18 04:00:00,cloudBase,0.210
1,1,2024-11-18 04:00:00,cloudCeiling,0.210
2,1,2024-11-18 04:00:00,cloudCover,92.000
3,1,2024-11-18 04:00:00,dewPoint,74.860
4,1,2024-11-18 04:00:00,evapotranspiration,0.005


In [8]:
# Join historical and forecast data so they can be plotted jointly.
data = pd.concat([historical_data, forecast_data], ignore_index=True)

In [9]:
# Define a line chart that is interactive. 
def plot_field(selected_field, selected_id):
    # Filter data for selected field and id
    mask = (data['field'] == selected_field) & (data['id'] == selected_id)
    filtered_data = data[mask]
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(filtered_data['datetime'], filtered_data['value'], marker='o')
    
    # Customize the plot
    plt.title(f'Time Series for Field: {selected_field}, ID: {selected_id}')
    plt.xlabel('DateTime')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    
    plt.show()



In [10]:
# Drop downs selectors for ID and Field of interest. 
field_dropdown = widgets.Dropdown(
    options=data['field'].unique(),
    description='Field:',
    style={'description_width': 'initial'}
)
id_dropdown = widgets.Dropdown(
    options=sorted(data['id'].unique()),
    description='ID:',
    style={'description_width': 'initial'}
)
interact(plot_field, selected_field=field_dropdown , selected_id=id_dropdown)

interactive(children=(Dropdown(description='Field:', options=('cloudBase', 'cloudCeiling', 'cloudCover', 'dewP…

<function __main__.plot_field(selected_field, selected_id)>